In [1]:
import json
import cv2
import imutils
import numpy as np
import random
import os

In [2]:
with open("cat_dog_annotations.json", "r") as f:
    data = json.load(f)
metadata = data["_via_img_metadata"]

In [3]:
images = []
annotations = []
classes = {}
n=1
annotation_id = 0
k = 1
class_id = 1
for key in metadata:
    image_data = metadata[key]
    image_name = image_data["filename"]
    image = cv2.imread(image_name)
    regions = image_data["regions"]
    angle = 5
    n-=1
    
    for j in range(72):
    
        scale_factor = np.random.uniform(0.6, 1)

        new_height = int(image.shape[0] * scale_factor)
        new_width = int(image.shape[1] * scale_factor)

        resized_image = cv2.resize(image, (new_width, new_height))
        resized_img_height, resized_img_width = resized_image.shape[:2]
        (centre_x, centre_y) = (resized_img_width//2, resized_img_height//2)
        
        contrast = 1
        brightness = random.uniform(1.1, 1.5)
        bright_resized = cv2.addWeighted(resized_image, brightness, resized_image, 0, contrast)
        
        Matrix = cv2.getRotationMatrix2D((centre_x,centre_y), angle*j, 1)
        cos = np.abs(Matrix[0, 0])
        sin = np.abs(Matrix[0, 1])
        new_RI_W = int((resized_img_height * sin) + (resized_img_width * cos))
        new_RI_H = int((resized_img_height * cos) + (resized_img_width * sin))
        Matrix[0, 2] += (new_RI_W / 2) - centre_x
        Matrix[1, 2] += (new_RI_H / 2) - centre_y
        
        rotated = cv2.warpAffine(bright_resized, Matrix, (new_RI_W, new_RI_H))
        h, w, _ = rotated.shape
        
        for i in regions:
            x = i["shape_attributes"]["x"]
            y = i["shape_attributes"]["y"]
            width = i["shape_attributes"]["width"]
            height = i["shape_attributes"]["height"]
            label = list(i["region_attributes"]["class"])[0]
            if label not in classes:
                classes[label] = class_id
                class_id += 1
            
            scaled_x = int(x * scale_factor)
            scaled_y = int(y * scale_factor)
            scaled_width = int(width * scale_factor)
            scaled_height = int(height * scale_factor)
            
            box_edges = np.array([[scaled_x, scaled_y],
                                  [scaled_x+scaled_width, scaled_y],
                                  [scaled_x+scaled_width, scaled_y+scaled_height],
                                  [scaled_x, scaled_y+scaled_height]])
            rotated_edges = np.dot(Matrix, np.vstack([box_edges.T, np.ones((1,4))]))
            x1 = int(np.min(rotated_edges[0]))
            y1 = int(np.min(rotated_edges[1]))
            x3 = int(np.max(rotated_edges[0]))
            y3 = int(np.max(rotated_edges[1]))
            rot_box_W = int(x3-x1)
            rot_box_H = int(y3-y1)
            
            cv2.rectangle(rotated, (x1,y1), (x3, y3), (0,0,255), 2)
            
            area = rot_box_H * rot_box_W
            
            category_id = classes[label]
            segmentations = [[x1, y1, x1+rot_box_W, y1, x3, y3, x1, y1+rot_box_H]]
            annotation = {
                "id": annotation_id,
                "category_id": category_id,
                "image_id": n,
                "segmentation": segmentations,
                "area": area,
                "bbox": [x1, y1, rot_box_W, rot_box_H],
                "iscrowd": 0
            }
            annotations.append(annotation)
            annotation_id+=1
        
        name = f"{k}_{j*angle}degree.png"
        images.append(
        {
            "id": n,
            "width": w,
            "height": h,
            "file_name": name,
            "license": 1,
            "data_captured": ""
        }
        )
        os.chdir('/media/ali-hassan/8A56FC4956FC3811/PF_DataScience/Evaluation_CV/cat_dog/resultant')
        cv2.imwrite(name, rotated)
        os.chdir('/media/ali-hassan/8A56FC4956FC3811/PF_DataScience/Evaluation_CV/cat_dog')
        n+=1
    n+=1
    k+=1

categories = [{"id": v, "name": k, "supercategory": "class"} for k, v in classes.items()]

coco = {
    "info": {
        "year": 2023,
        "version": "1",
        "description": "PF1",
        "contributor": "Ali_Hassan",
        "url": "",
        "date_created": "Sun Mar 26 2023 00:00:00 GMT+0500 (Pakistan Standard Time)"
    },
    "licenses": [
        {
            "id": 1,
            "name": "Unknown",
            "url": ""
        }
    ],
    "images": images,
    "annotations": annotations,
    "categories": categories
}

with open("coco_formatted_annotations.json", "w") as f:
    json.dump(coco, f)

print(n)
print(annotation_id)
print(classes)
print(categories)

505
720
{'dog': 1, 'cat': 2}
[{'id': 1, 'name': 'dog', 'supercategory': 'class'}, {'id': 2, 'name': 'cat', 'supercategory': 'class'}]
